In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hbalaji\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hbalaji\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
from transformers import pipeline
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
df=pd.read_csv('lyrics_10k.csv')
def clean_text(text, max_length=578):
    cleaned_text = ' '.join(text.split())
    #print(cleaned
    cleaned_text = cleaned_text.lower()
    cleaned_text = cleaned_text.translate(str.maketrans("", "", string.punctuation))
    #print(cleaned_text)
    cleaned_text = ''.join([i for i in cleaned_text if not i.isdigit()])
    #print(cleaned_text)
    cleaned_text = ''.join([i for i in cleaned_text if i.isalpha() or i.isspace()])
    #print(cleaned_text)
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(cleaned_text)
    cleaned_text = ' '.join([word for word in word_tokens if word.lower() not in stop_words])
    #print(cleaned_text)
    if len(cleaned_text) > max_length:
        cleaned_text = cleaned_text[:max_length]
    else:
        cleaned_text = cleaned_text.ljust(max_length)
    #print(cleaned_text)
    return cleaned_text


In [13]:
df['cleaned_lyrics'] = df['lyrics'].apply(clean_text)


In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the model and tokenizer
model_name = 'bhadresh-savani/distilbert-base-uncased-emotion'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# Initialize the emotion detection pipeline with truncation
emotion_detection = pipeline(
    'text-classification',
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    truncation=True
)

# Apply emotion detection to the cleaned lyrics
df['emotion'] = df['cleaned_lyrics'].apply(lambda x: emotion_detection(x)[0]['label'])
print(df[['cleaned_lyrics', 'emotion']])


                                         cleaned_lyrics  emotion
0     better home lyrics feel body breathing feel he...     fear
1     illusions lyrics people tell need help people ...    anger
2     touching ground lyrics swear touched saw heave...     love
3     nothing stopping lyrics nothing stopping stopp...      joy
4     shall believe lyrics come lay hands even lie s...  sadness
...                                                 ...      ...
8669  neon forest lyrics im sliding like lizard bell...     love
8670  harambe lyrics mafia billboard hitmakers yeah ...      joy
8671  lyrics seems like dreams like ive always could...      joy
8672  translations türkçe español selena gomez fun f...    anger
8673  translations türkçe português english english ...      joy

[8674 rows x 2 columns]


In [15]:
print(df.head())

   Unnamed: 0                               song_id  \
0           0  spotify:track:1NbzT9otocflZ9d8yvA54D   
1           1  spotify:track:1crTf4RvraCjcPoqsaT9hd   
2           2  spotify:track:4AGkBc4BiGLQNBdTUmUeF3   
3           3  spotify:track:4jtIeveEJPs8SxuVa3FV21   
4           4  spotify:track:5Dbr7Fy5VhSTXRbt2h3EYF   

                               artist_id                     song  \
0  spotify:artist:30TrHDLNCKQVTYWOn9QqOC         Better Than Home   
1  spotify:artist:4P0dddbxPil35MNN9G2MEX                Illusions   
2  spotify:artist:2sG4zTOLvjKG1PSoOyf5Ej      Touching the Ground   
3  spotify:artist:0c173mlxpT3dSFRgMO8XPh  Nothing Is Stopping You   
4  spotify:artist:4TKTii6gnOnUXQHyuo9JaD          I Shall Believe   

              artists  explicit  \
0       ['Beth Hart']     False   
1    ['Cypress Hill']      True   
2  ['Brandi Carlile']     False   
3        ['Big Sean']      True   
4     ['Sheryl Crow']     False   

                                           

In [21]:
emo_df=df[['song_id','artist_id','artists','emotion']]
df.to_csv('lyrics_emotion.csv',index=False)
df.to_csv('main.csv',index=False)